In [1]:
import onnx
import numpy as np
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType, quant_pre_process
from pathlib import Path
import json
import torch
import clip
import tempfile
import shutil

MODEL_DIR = Path("../../model")
FP32_PATH = MODEL_DIR / "clip-text-encoder.onnx"
COCO_ANN_JSON = Path("../../data/dataset/coco/annotations/captions_val2017.json")

# 加载数据
with open(COCO_ANN_JSON, 'r') as f:
    queries = [ann['caption'] for ann in json.load(f)['annotations'][:100]]
tokenized_data = [clip.tokenize([q]).numpy().astype(np.int64) for q in queries]

# ==================== 第一步：完整节点树分析 ====================
print("🔍 分析 ONNX 节点结构...")
model = onnx.load(str(FP32_PATH.parent / "clip-text-encoder-quant-pre.onnx"))

# 打印所有节点，按类型分类
node_types = {}
node_names_all = []

for node in model.graph.node:
    node_types.setdefault(node.op_type, []).append(node.name)
    node_names_all.append(node.name)

print(f"\n📊 节点类型统计:")
for op_type, nodes in sorted(node_types.items(), key=lambda x: -len(x[1])):
    print(f"   {op_type:15s}: {len(nodes):3d} 个")
    if len(nodes) <= 3:
        for n in nodes:
            print(f"      └─ {n}")

# 关键：找到所有 MatMul 节点
matmul_nodes = node_types.get('MatMul', [])
print(f"\n🎯 所有 MatMul 节点 ({len(matmul_nodes)} 个):")
for i, node_name in enumerate(matmul_nodes):
    print(f"   {i}: {node_name}")

# ==================== 第二步：按节点顺序逐个量化测试 ====================
print("\n\n🧪 按节点排序，逐个量化测试...")
print("="*70)

def get_similarity(model_path):
    """计算与 FP32 的相似度"""
    if not model_path.exists():
        return None

    try:
        sess = ort.InferenceSession(str(model_path),
                                    providers=['CPUExecutionProvider'])
        input_name = sess.get_inputs()[0].name

        similarities = []
        for token_batch in tokenized_data[:20]:  # 只用20条测试，加快速度
            output = sess.run(None, {input_name: token_batch})[0]
            output_norm = output / (np.linalg.norm(output, axis=-1, keepdims=True) + 1e-7)

            similarities.append(output_norm)

        return np.mean(np.concatenate(similarities))
    except Exception as e:
        print(f"   ❌ 错误: {e}")
        return None

# 获取 FP32 基准
sess_fp32 = ort.InferenceSession(str(FP32_PATH),
                                 providers=['CPUExecutionProvider'])
input_name = sess_fp32.get_inputs()[0].name
fp32_feats = []
for token_batch in tokenized_data[:20]:
    output = sess_fp32.run(None, {input_name: token_batch})[0]
    output_norm = output / (np.linalg.norm(output, axis=-1, keepdims=True) + 1e-7)
    fp32_feats.append(output_norm)
fp32_feats = np.concatenate(fp32_feats)

def get_cos_sim(model_path):
    """更精准的相似度计算"""
    if not model_path.exists():
        return None

    try:
        sess = ort.InferenceSession(str(model_path),
                                    providers=['CPUExecutionProvider'])
        input_name = sess.get_inputs()[0].name

        int8_feats = []
        for token_batch in tokenized_data[:20]:
            output = sess.run(None, {input_name: token_batch})[0]
            output_norm = output / (np.linalg.norm(output, axis=-1, keepdims=True) + 1e-7)
            int8_feats.append(output_norm)
        int8_feats = np.concatenate(int8_feats)

        return np.mean(np.sum(fp32_feats * int8_feats, axis=-1))
    except:
        return None

# ==================== 关键策略：逐个 MatMul 添加排除 ====================
print("\n策略 A: 逐个排除 MatMul 节点\n")

results_matmul = []
temp_dir = MODEL_DIR / "temp_tests"
temp_dir.mkdir(exist_ok=True)

# 先测试全量化
test_path = temp_dir / "all_quantized.onnx"
quantize_dynamic(
    str(FP32_PATH.parent / "clip-text-encoder-quant-pre.onnx"),
    str(test_path),
    weight_type=QuantType.QInt8,
    nodes_to_exclude=['node_GatherND_1006']  # 只排除 GatherND
)
sim_baseline = get_cos_sim(test_path)
print(f"🔴 全量化 (仅排除 GatherND): {sim_baseline:.4f}")
results_matmul.append(('全量化', sim_baseline))
test_path.unlink()

# 逐个排除 MatMul
for i, matmul_name in enumerate(matmul_nodes):
    exclude_list = ['node_GatherND_1006'] + matmul_nodes[:i+1]

    test_path = temp_dir / f"exclude_matmul_{i}.onnx"
    quantize_dynamic(
        str(FP32_PATH.parent / "clip-text-encoder-quant-pre.onnx"),
        str(test_path),
        weight_type=QuantType.QInt8,
        nodes_to_exclude=exclude_list
    )

    sim = get_cos_sim(test_path)
    results_matmul.append((f"排除前 {i+1} 个 MatMul", sim))

    print(f"   排除前 {i+1:2d} 个 MatMul (含 {matmul_name}): {sim:.4f}")

    test_path.unlink()

# ==================== 策略 B: 逐个 Softmax 排除 ====================
print("\n策略 B: 逐个排除 Softmax 节点\n")

softmax_nodes = node_types.get('Softmax', [])
print(f"Softmax 节点数: {len(softmax_nodes)}\n")

results_softmax = []

# 先排除所有 Softmax
exclude_list = ['node_GatherND_1006'] + softmax_nodes
test_path = temp_dir / "exclude_all_softmax.onnx"
quantize_dynamic(
    str(FP32_PATH.parent / "clip-text-encoder-quant-pre.onnx"),
    str(test_path),
    weight_type=QuantType.QInt8,
    nodes_to_exclude=exclude_list
)
sim_all_softmax = get_cos_sim(test_path)
print(f"🟡 排除所有 Softmax: {sim_all_softmax:.4f}")
results_softmax.append(('排除所有 Softmax', sim_all_softmax))
test_path.unlink()

# ==================== 策略 C: 贪心搜索 ====================
print("\n策略 C: 贪心搜索（找最敏感的节点）\n")

base_exclude = ['node_GatherND_1006', 'node_matmul']  # 基础排除
current_exclude = base_exclude.copy()
current_sim = sim_baseline

remaining_nodes = [n for n in matmul_nodes if n not in current_exclude] + \
                  [n for n in softmax_nodes if n not in current_exclude]

print(f"初始相似度: {current_sim:.4f}\n")

greedy_results = []

for iteration in range(min(10, len(remaining_nodes))):
    best_sim = current_sim
    best_node = None

    print(f"第 {iteration+1} 轮: 尝试添加 {len(remaining_nodes)} 个节点...")

    for test_node in remaining_nodes[:5]:  # 只测试前5个，加快速度
        test_exclude = current_exclude + [test_node]
        test_path = temp_dir / f"greedy_{iteration}_{test_node[:20]}.onnx"

        try:
            quantize_dynamic(
                str(FP32_PATH.parent / "clip-text-encoder-quant-pre.onnx"),
                str(test_path),
                weight_type=QuantType.QInt8,
                nodes_to_exclude=test_exclude
            )

            sim = get_cos_sim(test_path)
            print(f"   + {test_node[:50]:50s}: {sim:.4f}", end="")

            if sim > best_sim:
                print(" ✨ 最好!")
                best_sim = sim
                best_node = test_node
            else:
                print()

            test_path.unlink()
        except:
            pass

    if best_node is None:
        print(f"   无法改进，停止搜索\n")
        break

    print(f"✅ 第 {iteration+1} 轮最优: +{best_node} → {best_sim:.4f}\n")

    current_exclude.append(best_node)
    current_sim = best_sim
    remaining_nodes.remove(best_node)

    greedy_results.append((f"第{iteration+1}轮", current_sim, best_node))

# ==================== 总结 ====================
print("\n" + "="*70)
print("📊 诊断总结")
print("="*70)

if results_matmul:
    best_matmul = max(results_matmul, key=lambda x: x[1])
    print(f"\n🥇 MatMul 策略最优: {best_matmul[0]}")
    print(f"   相似度: {best_matmul[1]:.4f}")

if results_softmax:
    best_softmax = max(results_softmax, key=lambda x: x[1])
    print(f"\n🥇 Softmax 策略最优: {best_softmax[0]}")
    print(f"   相似度: {best_softmax[1]:.4f}")

if greedy_results:
    best_greedy = max(greedy_results, key=lambda x: x[1])
    print(f"\n🥇 贪心搜索最优: {best_greedy[0]}")
    print(f"   相似度: {best_greedy[1]:.4f}")
    print(f"   关键节点: {best_greedy[2]}")

print("\n" + "="*70)

# 清理
shutil.rmtree(temp_dir)

🔍 分析 ONNX 节点结构...

📊 节点类型统计:
   Reshape        : 134 个
   Transpose      :  74 个
   Add            :  73 个
   MatMul         :  61 个
   Mul            :  50 个
   Gather         :  37 个
   Slice          :  36 个
   Concat         :  30 个
   LayerNormalization:  25 个
   Unsqueeze      :  14 个
   Shape          :  13 个
   Squeeze        :  13 个
   Softmax        :  12 个
   Gemm           :  12 个
   Sigmoid        :  12 个
   Range          :   1 个
      └─ node_arange
   ArgMax         :   1 个
      └─ node_argmax
   GatherND       :   1 个
      └─ node_GatherND_1006

🎯 所有 MatMul 节点 (61 个):
   0: node_MatMul_3
   1: node_MatMul_74
   2: node_scaled_dot_product_attention
   3: node_MatMul_88
   4: node_MatMul_91
   5: node_MatMul_93
   6: node_MatMul_158
   7: node_scaled_dot_product_attention_1
   8: node_MatMul_171
   9: node_MatMul_173
   10: node_MatMul_175
   11: node_MatMul_240
   12: node_scaled_dot_product_attention_2
   13: node_MatMul_253
   14: node_MatMul_255
   15: node_MatMul_